In [4]:
import requests
from investpy.utils.extra import random_user_agent
#from tessa.rate_limiter import rate_limit

import datetime
from bs4 import BeautifulSoup
from tqdm import tqdm

In [5]:
def getInvesting(ticker):

    target_address = 'https://www.google.com/search?q='+ticker+'+investing'
    html = requests.get(target_address)
    soup = BeautifulSoup(html.text, 'html.parser')
    posts = soup.select('.kCrYT a ')
    #for chooes first herf 
    href = str(posts[0].attrs['href'])
    #then href is 'https:/www.investing/equities/stock&~~~~~~ 
    #so 1.find  start-index : index('equities') , end-index : index('&')
    # and slicing href[start : end]
    start = href.find('equities')
    end = href.find('&')
    # print(href[start:end])

    return "https://www.investing.com/" + href[start:end] + '-commentary'

In [45]:
url = getInvesting('TSLA') + '/' + str(2)
        
head = {
    "User-Agent": random_user_agent(),
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "text/html",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
}

rate_limit("investing")
req = requests.get(url, headers=head)
soup = BeautifulSoup(req.text, 'html.parser')
items = soup.select_one("#comments > ul")
link = soup.find_all('link', rel='canonical')

In [46]:
link[0]['href'] == getInvesting('TSLA') + '/' + str(2)

True

In [2]:
getInvesting('TSLA')

NameError: name 'requests' is not defined

In [35]:
getInvesting('TSLA') + '/' + str(2)

'https://www.investing.com/equities/tesla-motors/2'

In [36]:
def crawling(corp, writer):
    i = 1

    while True:
        i += 1
        
        url = getInvesting(corp) + '/' + str(i)
        
        head = {
            "User-Agent": random_user_agent(),
            "X-Requested-With": "XMLHttpRequest",
            "Accept": "text/html",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
        }

        rate_limit("investing")
        req = requests.get(url, headers=head)
        soup = BeautifulSoup(req.text, 'html.parser')
        items = soup.select_one("#comments > ul")
        link = soup.find_all('link', rel='canonical')
        
        print(link)
        
        if link[0]['href'] == getInvesting(corp) + '/' + 2:
            break
        
        
        tolerance = 100
        while items is None and tolerance > 0:
            with open("error.html", "w") as f:
                f.write(req.text)
            
            rate_limit("investing")
            req = requests.get(url, headers=head)
            soup = BeautifulSoup(req.text, 'html.parser')
            items = soup.select_one("#comments > ul")
            tolerance -= 1
        
        if tolerance == 0:
            print("end: ", datetime.datetime.now())
            break
        
        request_date = datetime.datetime.strptime(req.headers["Date"],
                                                  "%a, %d %b %Y %H:%M:%S GMT")

        for item in items:
            comment = item.select_one("div.comment_content__AvzPV").text
            comment_date = item.select_one("span[data-test='comment-date']").text

            if comment_date.endswith("minute ago"):
                delta = int(comment_date.replace("minute ago", ""))
                comment_date = request_date - datetime.timedelta(minutes=delta)
                comment_date = comment_date.strftime("%b %d, %Y, %H:%M")
            elif comment_date.endswith("minutes ago"):
                delta = int(comment_date.replace("minutes ago", ""))
                comment_date = request_date - datetime.timedelta(minutes=delta)
                comment_date = comment_date.strftime("%b %d, %Y, %H:%M")
            elif comment_date.endswith("hour ago"):
                delta = int(comment_date.replace("hour ago", ""))
                comment_date = request_date - datetime.timedelta(minutes=delta)
                comment_date = comment_date.strftime("%b %d, %Y, %H:%M")
            elif comment_date.endswith("hours ago"):
                delta = int(comment_date.replace("hours ago", ""))
                comment_date = request_date - datetime.timedelta(minutes=delta)
                comment_date = comment_date.strftime("%b %d, %Y, %H:%M")
                
            line = f"{comment}\t{comment_date}\n"
                
            writer.write(line)

In [25]:
if __name__ == "__main__":
    
    f = open(f'corporate.txt', 'r')

    while True:
        corp = f.readline()
        corp = str(corp)
        
        if not line: break
    
        with open(f"{corp}.csv", "w") as out_file:
            crawling(corp, writer=out_file)

In [12]:
f = open(f'corporate.txt', 'r')

while True:
    line = f.readline()
    if not line: break
    print(line)

MMM

AOS

ABT

ABBV

ABMD

ACN

ATVI

ADM

ADBE

ADP

AAP

AES

AFL

A

AIG

APD

AKAM

ALK

ALB

ARE

ALGN

ALLE

LNT

ALL

GOOGL

GOOG

MO

AMZN

AMCR

AMD

AEE

AAL

AEP

AXP

AMT

AWK

AMP

ABC

AME

AMGN

APH

ADI

ANSS

ANTM

AON

APA

AAPL

AMAT

APTV

ANET

AIZ

T

ATO

ADSK

AZO

AVB

AVY

BKR

BALL

BAC

BBWI

BAX

BDX

WRB

BRKB

BBY

BIO

TECH

BIIB

BLK

BK

BA

BKNG

BWA

BXP

BSX

BMY

AVGO

BR

BRO

BFB

CHRW

CDNS

CZR

CPT

CPB

COF

CAH

KMX

CCL

CARR

CTLT

CAT

CBOE

CBRE

CDW

CE

CNC

CNP

CDAY

CF

CRL

SCHW

CHTR

CVX

CMG

CB

CHD

CI

CINF

CTAS

CSCO

C

CFG

CTXS

CLX

CME

CMS

KO

CTSH

CL

CMCSA

CMA

CAG

COP

ED

STZ

CEG

COO

CPRT

GLW

CTVA

COST

CTRA

CCI

CSX

CMI

CVS

DHI

DHR

DRI

DVA

DE

DAL

XRAY

DVN

DXCM

FANG

DLR

DFS

DISH

DIS

DG

DLTR

D

DPZ

DOV

DOW

DTE

DUK

DRE

DD

DXC

EMN

ETN

EBAY

ECL

EIX

EW

EA

EMR

ENPH

ETR

EOG

EPAM

EFX

EQIX

EQR

ESS

EL

ETSY

RE

EVRG

ES

EXC

EXPE

EXPD

EXR

XOM

FFIV

FDS

FAST

FRT



In [33]:
pbar = tqdm()
i = 1

while True:
    i += 1    

    pbar.update(i)
    
    if i == 100:
        break
        
pbar.close()



5049it [00:00, 11771562.48it/s]
